In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [40.3 kB]
Get:12 http://ppa.launchpad.net/graph

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Analysis").getOrCreate()

In [3]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

In [5]:
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3DT59XH7HXR9K|          5|            0|          0|   N|                Y|
|R1LFS11BNASSU8|          5|            0|          1|   N|                Y|
|R296RT05AG0AF6|          5|            0|          0|   N|                Y|
|R3V37XDZ7ZCI3L|          5|            0|          1|   N|                Y|
|R14GU232NQFYX2|          5|            0|          0|   N|                Y|
| RZQH4V7L2O1PL|          1|            1|          1|   N|                Y|
|R1F8JMOSPJ3KO7|          5|            0|          0|   N|                Y|
|R1ZISGY2BWW4Z5|          5|            0|          0|   N|                Y|
|R17PW4I3AE5WZW|          5|            0|          0|   N|                Y|
|R3D93G1KTP6A8P|          3|            0|          0|   N|     

In [6]:
# 1) Filter dataframe to retrieve rows where total_votes >= 20
vine_updated_df = vine_df.filter('total_votes>=20')
vine_updated_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R28RB82UG4RDD5|          5|           20|         20|   N|                Y|
|R3FB6BERWPEIJP|          4|           40|         43|   N|                Y|
|R1D4Z38STRDQXK|          5|           53|         56|   N|                Y|
|R1XMWJZICINIFX|          3|           20|         21|   N|                Y|
|R20QKY1GABXFLM|          1|          272|        297|   N|                Y|
|R3GZ7CK2O1PPG0|          2|           10|         30|   N|                Y|
|R328FA1E6FY3F5|          5|           17|         20|   N|                N|
|R3DH22AA5WGLLS|          5|           30|         30|   N|                N|
|R1E0E5EFZSLJCS|          1|           66|         80|   N|                Y|
|R1TXZQWEHYWEWN|          2|           48|         51|   N|     

In [7]:
# filter new dataframe to retrieve rows where the number of helpful_votes / total_votes >= 50%
vine_votes_df = vine_updated_df.filter('helpful_votes / total_votes >= .5')
vine_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R28RB82UG4RDD5|          5|           20|         20|   N|                Y|
|R3FB6BERWPEIJP|          4|           40|         43|   N|                Y|
|R1D4Z38STRDQXK|          5|           53|         56|   N|                Y|
|R1XMWJZICINIFX|          3|           20|         21|   N|                Y|
|R20QKY1GABXFLM|          1|          272|        297|   N|                Y|
|R328FA1E6FY3F5|          5|           17|         20|   N|                N|
|R3DH22AA5WGLLS|          5|           30|         30|   N|                N|
|R1E0E5EFZSLJCS|          1|           66|         80|   N|                Y|
|R1TXZQWEHYWEWN|          2|           48|         51|   N|                Y|
| RQQAI8YL3UCY2|          5|           23|         25|   N|     

In [8]:
# Retrieve rows where the review was part of the Vine program (paid)
vine_paid_df = vine_votes_df.filter(vine_votes_df['vine'] == 'Y')
vine_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1Z71RW4J9IK93|          5|           20|         22|   Y|                N|
|R3FVB5QI11KI9Q|          4|          192|        200|   Y|                N|
|R2G027YBMVXV6Y|          5|           39|         48|   Y|                N|
|R1QGBAN7BMGWRR|          5|          121|        129|   Y|                N|
|R2NH2AU7XL9ZDZ|          3|           18|         20|   Y|                N|
|R2YVVJ9NOPNX50|          4|           36|         41|   Y|                N|
|R1XH1LK1FWX3OS|          4|          214|        238|   Y|                N|
|R38LSQ71G2IZGS|          5|           26|         29|   Y|                N|
|R2FLITQVKWXDF4|          3|           26|         34|   Y|                N|
|R25LMMZF3DJTWY|          2|           18|         21|   Y|     

In [9]:
# Retrieve rows where the review was not part of the Vine program (unpaid)
vine_unpaid_df = vine_votes_df.filter(vine_votes_df['vine'] == 'N')
vine_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R28RB82UG4RDD5|          5|           20|         20|   N|                Y|
|R3FB6BERWPEIJP|          4|           40|         43|   N|                Y|
|R1D4Z38STRDQXK|          5|           53|         56|   N|                Y|
|R1XMWJZICINIFX|          3|           20|         21|   N|                Y|
|R20QKY1GABXFLM|          1|          272|        297|   N|                Y|
|R328FA1E6FY3F5|          5|           17|         20|   N|                N|
|R3DH22AA5WGLLS|          5|           30|         30|   N|                N|
|R1E0E5EFZSLJCS|          1|           66|         80|   N|                Y|
|R1TXZQWEHYWEWN|          2|           48|         51|   N|                Y|
| RQQAI8YL3UCY2|          5|           23|         25|   N|     

In [32]:
# determine total number of reviews
total_reviews = vine_votes_df.count()
total_reviews

99046

In [33]:
# determine number of 5-star reviews
five_star_reviews = vine_votes_df.filter('star_rating==5').count()
five_star_reviews

46367

In [34]:
# total number of vine reviews
paid_reviews = vine_paid_df.count()
paid_reviews


1207

In [37]:
# total number of non-vine reviews
unpaid_reviews = vine_unpaid_df.count()
unpaid_reviews


97839

In [35]:
# total number of 5-star vine reviews
paid_five_star_reviews = vine_paid_df.filter('star_rating==5').count()
paid_five_star_reviews

509

In [36]:
# total number of 5-star non-vine reviews
unpaid_five_star_reviews = vine_unpaid_df.filter('star_rating==5').count()
unpaid_five_star_reviews

45858

In [39]:
# % of unpaid 5 star reviews compared to all unpaid reviews
unpaid_percentage = unpaid_five_star_reviews/unpaid_reviews*100
unpaid_percentage

46.87087971054488

In [41]:
# % of paid 5 star reviews compared to all paid reviews
paid_percentage = paid_five_star_reviews/paid_reviews*100
paid_percentage

42.17067108533554

In [43]:
# % of non-vine 5 star reviews compared to all reviews
unpaid_total_percentage = unpaid_five_star_reviews/total_reviews*100
unpaid_total_percentage

46.299699129697316

In [45]:
# % of vine 5 star reviews compaired to all reviews
paid_total_percentage = paid_five_star_reviews/total_reviews*100
paid_total_percentage

0.5139026311007007

In [48]:
# % of unpaid 5 star reviews compared to all 5 star reviews
unpaid_five_star_percent = unpaid_five_star_reviews/five_star_reviews*100
unpaid_five_star_percent

98.90223650441047

In [49]:
# % of paid 5 star reviews compared to all 5 star reviews
paid_five_star_percent = paid_five_star_reviews/five_star_reviews*100
paid_five_star_percent

1.0977634955895357